In [ ]:
import BAC0
import time
import os, sys
import paho.mqtt.client as mqtt
import uuid
import sqlite3
import datetime

conn = sqlite3.connect('C:\\Users\\PARROT\\Desktop\\BACnetData.db')
c = conn.cursor()
try:
    c.execute("CREATE TABLE Data (id integer PRIMARY KEY, TimeStamp timestamp, VAR_NAME TEXT, Value TEXT)")
    conn.commit()
except:
    pass

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [ ]:
def get_bac0_device(device, network):
    print("Get BAC0 device")
    name, vendor, address, device_id = device
    return BAC0.device(address, device_id, network, poll=0,
    object_list=None)

In [ ]:
def main():

    # BAC0 Setup
    bacnet = BAC0.connect()
    bacnet.discover()
    device = bacnet.devices[0]
    bac0_device = get_bac0_device(device, bacnet)
    
    # Main Loop of sending data
    while True:
        points = bac0_device.points
        blockPrint()
        print(points)
        enablePrint()

        for idx, point in enumerate(points):
            timestamp = datetime.datetime.now()
            c.execute("INSERT INTO Data (TimeStamp, VAR_NAME, Value) VALUES (?, ?, ?)", (timestamp, point.properties.name, point.lastValue))
            conn.commit()
            
        # Wait for 10 seconds before sending updated data
        time.sleep(4)

In [ ]:
if __name__ == "__main__":
    main()